In [1]:
import os 
os.chdir("/workspace")
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip

# import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
import math
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.distributions import Categorical
import models.pos_encoding as pos_encoding
import clip

warnings.filterwarnings('ignore')

In [3]:
import easydict

args = easydict.EasyDict({
    'dataname': 't2m',
    'batch_size': 16,
    'fps': [20],
    'seq_len': 64,
    'total_iter': 100000,
    'warm_up_iter': 1000,
    'lr': 0.0001,
    'lr_scheduler': [150000],
    'gamma': 0.05,
    'weight_decay': 1e-6,
    'decay_option': 'all',
    'optimizer': 'adamw',
    'code_dim': 512,
    'nb_code': 512,
    'mu': 0.99,
    'down_t': 2,
    'stride_t': 2,
    'width': 512,
    'depth': 3,
    'dilation_growth_rate': 3,
    'output_emb_width': 512,
    'vq_act': 'relu',
    'block_size': 51,
    'embed_dim_gpt': 1024,
    'clip_dim': 512,
    'num_layers': 9,
    'n_head_gpt': 16,
    'ff_rate': 4,
    'drop_out_rate': 0.1,
    'quantizer': 'ema_reset',
    'quantbeta': 1.0,
    'resume_pth': "output_vqfinal/VQ-VAE/reset/net_last.pth",
    'resume_trans': None,
    'out_dir': 'output_GPT_Final/',
    'exp_name': 'exp_debug',
    'vq_name': 'exp_debug',
    'print_iter': 200,
    'eval_iter': 10000,
    'seed': 123,
    'if_maxtest': False,
    'pkeep': 0.5
})

In [4]:
args.out_dir = os.path.join(args.out_dir, f'{args.exp_name}')
args.vq_dir= os.path.join("./dataset/KIT-ML" if args.dataname == 'kit' else "./dataset/HumanML3D", f'{args.vq_name}')
os.makedirs(args.out_dir, exist_ok = True)
os.makedirs(args.vq_dir, exist_ok = True)

In [5]:
class CausalCrossConditionalSelfAttention(nn.Module):

    def __init__(self, embed_dim=512, block_size=16, n_head=8, drop_out_rate=0.1):
        super().__init__()
        assert embed_dim % 8 == 0
        # key, query, value projections for all heads
        self.key = nn.Linear(embed_dim, embed_dim)
        self.query = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)

        self.attn_drop = nn.Dropout(drop_out_rate)
        self.resid_drop = nn.Dropout(drop_out_rate)

        self.proj = nn.Linear(embed_dim, embed_dim)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("mask", torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size))
        self.n_head = n_head

    def forward(self, x):
        B, T, C = x.size() 

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        k = self.key(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = self.query(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = self.value(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        
        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))

        #mask 값이 0인 부분을 -inf로 채우기 
        att = att.masked_fill(self.mask[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)

        #Scaled Dot-Product Attention을 계산
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        #(B,  T,  m_ebed_dim)
        # output projection
        y = self.resid_drop(self.proj(y))
        return y


In [6]:
class Block(nn.Module):

    def __init__(self, embed_dim=512, block_size=16, n_head=8, drop_out_rate=0.1, fc_rate=4):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.attn = CausalCrossConditionalSelfAttention(embed_dim, block_size, n_head, drop_out_rate)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, fc_rate * embed_dim),
            nn.GELU(),
            nn.Linear(fc_rate * embed_dim, embed_dim),
            nn.Dropout(drop_out_rate),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

In [7]:
class CrossCondTransBase(nn.Module):

    def __init__(self, 
                num_vq=1024, 
                embed_dim=512,
                clip_dim=512, 
                block_size=16, 
                num_layers=2, 
                n_head=8, 
                drop_out_rate=0.1, 
                fc_rate=4):
        super().__init__()
        
        self.tok_emb = nn.Embedding(num_vq + 2, embed_dim)
        self.cond_emb = nn.Linear(clip_dim, embed_dim)
        self.pos_embedding = nn.Embedding(block_size, embed_dim)
        self.drop = nn.Dropout(drop_out_rate)
        # transformer block
        self.blocks = nn.Sequential(*[Block(embed_dim, block_size, n_head, drop_out_rate, fc_rate) for _ in range(num_layers)])
        self.pos_embed = pos_encoding.PositionEmbedding(block_size, embed_dim, 0.0, False)

        self.block_size = block_size

        self.apply(self._init_weights)

    def get_block_size(self):
        return self.block_size

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def forward(self, idx, clip_feature):
        if len(idx) == 0:
            token_embeddings = self.cond_emb(clip_feature).unsqueeze(1)
        else:
            print(idx.size())
            b, t = idx.size()
            assert t <= self.block_size, "Cannot forward, model block size is exhausted."
            # forward the Trans model
            #idx = motion_idx, feat_clip_text=>()
            token_embeddings = self.tok_emb(idx)
            print("idx size :" , idx.shape)
            print("token ebedding : ", token_embeddings.shape)
            print("cond_emd shape", self.cond_emb(clip_feature).unsqueeze(1).shape)
            token_embeddings = torch.cat([self.cond_emb(clip_feature).unsqueeze(1), token_embeddings], dim=1)
            print("token ebedding_cat : ", token_embeddings.shape)
        
        x = self.pos_embed(token_embeddings)
        print("pos",x.shape)
        
        x = self.blocks(x)
        print("block",x.shape)

        return x


In [8]:
class CrossCondTransHead(nn.Module):

    def __init__(self, 
                num_vq=1024, 
                embed_dim=512, 
                block_size=16, 
                num_layers=2, 
                n_head=8, 
                drop_out_rate=0.1, 
                fc_rate=4):
        super().__init__()

        self.blocks = nn.Sequential(*[Block(embed_dim, block_size, n_head, drop_out_rate, fc_rate) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_vq + 1, bias=False)
        self.block_size = block_size

        self.apply(self._init_weights)

    def get_block_size(self):
        return self.block_size

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self, x):
        x = self.blocks(x)
        print("x : ", x.shape)
        x = self.ln_f(x)
        print("x_layerNorm : ", x.shape)
        logits = self.head(x)
        print("logits : ", logits.shape)
        return logits


In [9]:
class Text2Motion_Transformer(nn.Module):

    def __init__(self, 
                num_vq=1024, 
                embed_dim=512, 
                clip_dim=512, 
                block_size=16, 
                num_layers=2, 
                n_head=8, 
                drop_out_rate=0.1, 
                fc_rate=4):
        super().__init__()
        self.trans_base = CrossCondTransBase(num_vq, embed_dim, clip_dim, block_size, num_layers, n_head, drop_out_rate, fc_rate)
        self.trans_head = CrossCondTransHead(num_vq, embed_dim, block_size, num_layers, n_head, drop_out_rate, fc_rate)
        self.block_size = block_size
        self.num_vq = num_vq

    def get_block_size(self):
        return self.block_size

    def forward(self, idxs, clip_feature):
        feat = self.trans_base(idxs, clip_feature)
        logits = self.trans_head(feat)
        return logits

    def sample(self, clip_feature, if_categorial=False):
        for k in range(self.block_size):
            if k == 0:
                x = []
            else:
                x = xs
            logits = self.forward(x, clip_feature)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            if if_categorial:
                dist = Categorical(probs)
                idx = dist.sample()
                if idx == self.num_vq:
                    break
                idx = idx.unsqueeze(-1)
            else:
                _, idx = torch.topk(probs, k=1, dim=-1)
                if idx[0] == self.num_vq:
                    break
            # append to the sequence and continue
            if k == 0:
                xs = idx
            else:
                xs = torch.cat((xs, idx), dim=1)
            
            if k == self.block_size - 1:
                return xs[:, :-1]
        return xs

In [10]:
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

In [11]:
trans_encoder = Text2Motion_Transformer(num_vq=args.nb_code, 
                                embed_dim=args.embed_dim_gpt, 
                                clip_dim=args.clip_dim, 
                                block_size=args.block_size, 
                                num_layers=args.num_layers, 
                                n_head=args.n_head_gpt, 
                                drop_out_rate=args.drop_out_rate, 
                                fc_rate=args.ff_rate)

trans_encoder.train()
trans_encoder.cuda()

Text2Motion_Transformer(
  (trans_base): CrossCondTransBase(
    (tok_emb): Embedding(514, 1024)
    (cond_emb): Linear(in_features=512, out_features=1024, bias=True)
    (pos_embedding): Embedding(51, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (ln1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CausalCrossConditionalSelfAttention(
          (key): Linear(in_features=1024, out_features=1024, bias=True)
          (query): Linear(in_features=1024, out_features=1024, bias=True)
          (value): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_drop): Dropout(p=0.1, inplace=False)
          (resid_drop): Dropout(p=0.1, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (mlp): Sequential(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
      

In [12]:
train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

train_loader = dataset_TM_train.DATALoader(args.dataname, args.batch_size, args.nb_code, args.vq_name, unit_length=2**args.down_t)
train_loader_iter = dataset_TM_train.cycle(train_loader)


100%|██████████| 23384/23384 [00:03<00:00, 6704.16it/s]


64
4


100%|██████████| 23384/23384 [00:03<00:00, 7469.16it/s]


In [13]:
batch = next(train_loader_iter)

In [14]:
clip_text, m_tokens, m_tokens_len = batch
m_tokens.shape
m_tokens, m_tokens_len = m_tokens.cuda(), m_tokens_len.cuda()
bs = m_tokens.shape[0]
m_tokens.shape

torch.Size([16, 51])

In [19]:
target = m_tokens    # (bs, 51)
print(target.shape)
target = target.cuda()
text = clip.tokenize(clip_text, truncate=True).cuda()
print(text)
print(text.shape)

torch.Size([16, 51])
tensor([[49406,   518,  2533,  ...,     0,     0,     0],
        [49406,   518,  5706,  ...,     0,     0,     0],
        [49406,   320,  2533,  ...,     0,     0,     0],
        ...,
        [49406,   320,  2533,  ...,     0,     0,     0],
        [49406,   320,  2533,  ...,     0,     0,     0],
        [49406,   518,  2533,  ...,     0,     0,     0]], device='cuda:0',
       dtype=torch.int32)
torch.Size([16, 77])


In [20]:
feat_clip_text = clip_model.encode_text(text).float()
input_index = target[:,:-1]
print(input_index.shape)
print(input_index)

torch.Size([16, 50])
tensor([[390, 144, 282, 301, 452, 260, 102, 294, 480, 233, 267, 275,  12, 419,
         357, 135, 452, 203,  89, 330, 113, 409, 194, 438, 384, 412, 321, 321,
          52,  71, 257, 472, 324, 111, 381, 267, 113, 324, 159, 219, 393, 468,
         471, 234, 275, 412, 393, 310, 316, 512],
        [165, 175, 224, 319, 319, 319, 319, 319, 224, 224,  31, 473, 473, 473,
         291, 405, 354, 175, 224,  31,  37, 291, 291, 405, 354, 175, 224, 224,
          37, 473, 473, 473, 473, 473,  31, 175, 175, 475,  17, 319, 319, 319,
         319, 319, 319, 319, 319, 224, 165, 512],
        [402, 402, 402, 402, 127, 328, 319,  53, 449, 258,  52, 131, 248, 489,
          11, 185, 454,  33, 206,  97,  97,  97, 512, 513, 513, 513, 513, 513,
         513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513,
         513, 513, 513, 513, 513, 513, 513, 513],
        [273, 273, 317, 168, 168, 273, 317, 159, 328, 254, 300, 282, 220, 350,
         453,  11,  28, 318, 184, 448, 

In [17]:
# 베르누이 분포에서 난수 0,1로 그림
#([128, 50])
mask = torch.bernoulli(args.pkeep * torch.ones(input_index.shape, device=input_index.device))
mask = mask.round().to(dtype=torch.int64)
r_indices = torch.randint_like(input_index, args.nb_code)
print(r_indices.shape)
#mask 값이 1인 위치에는 input_index의 값을 넣고 0에서는 r_indices
#노이즈를 추가하는 효과
a_indices = mask*input_index+(1-mask)*r_indices
a_indices.shape

torch.Size([16, 50])


torch.Size([16, 50])

In [18]:
cls_pred = trans_encoder(a_indices, feat_clip_text)   
print(cls_pred.shape) 
cls_pred = cls_pred.contiguous()

torch.Size([16, 50])
idx size : torch.Size([16, 50])
token ebedding :  torch.Size([16, 50, 1024])
cond_emd shape torch.Size([16, 1, 1024])
token ebedding_cat :  torch.Size([16, 51, 1024])
pos torch.Size([16, 51, 1024])
block torch.Size([16, 51, 1024])
x :  torch.Size([16, 51, 1024])
x_layerNorm :  torch.Size([16, 51, 1024])
logits :  torch.Size([16, 51, 513])
torch.Size([16, 51, 513])


In [21]:
cls_pred.shape

torch.Size([16, 51, 513])

In [22]:
loss_ce = torch.nn.CrossEntropyLoss()

In [23]:
loss_cls = 0.0
loss_cls += loss_ce(cls_pred[0][:m_tokens_len[0] + 1], target[0][:m_tokens_len[0] + 1])

In [28]:
m_tokens_len[2]

tensor(22, device='cuda:0')

In [29]:
cls_pred[2][:m_tokens_len[2] + 1].shape

torch.Size([23, 513])

In [24]:
loss_cls

tensor(6.5367, device='cuda:0', grad_fn=<AddBackward0>)

In [48]:
cls_pred[0][:m_tokens_len[0] + 1].shape

torch.Size([50, 513])

In [46]:
probs = torch.softmax(cls_pred[0][:m_tokens_len[0] + 1], dim=-1)

probs.shape

torch.Size([50, 513])

In [31]:
dist = Categorical(probs)
cls_pred_index = dist.sample()

In [47]:
len(cls_pred_index)

50

In [35]:
right_num = 0
right_num += (cls_pred_index.flatten(0) == target[0][:m_tokens_len[0] + 1].flatten(0)).sum().item()

In [42]:
target[0][:m_tokens_len[0] + 1]

tensor([390, 144, 282, 301, 452, 260, 102, 294, 480, 233, 267, 275,  12, 419,
        357, 135, 452, 203,  89, 330, 113, 409, 194, 438, 384, 412, 321, 321,
         52,  71, 257, 472, 324, 111, 381, 267, 113, 324, 159, 219, 393, 468,
        471, 234, 275, 412, 393, 310, 316, 512], device='cuda:0')

In [49]:
_, cls_pred_index = torch.max(probs, dim=-1)

In [50]:
cls_pred_index

tensor([449, 357,  41, 233,  89,  41, 202, 476, 476, 233, 233, 229,  15,  41,
         50,  41,  50,  50,  15,  50,  31,  50,  50, 181, 229,  50, 476, 220,
         68,  50,  50,  50,  50, 229,  15,  15,  98,  15, 476, 476, 110,  50,
         50,  50,  50,  50, 476,  50, 304, 466], device='cuda:0')

In [36]:
right_num

0

In [37]:
cls_pred_index.flatten(0) 

tensor([ 23,  96,  54, 378, 237, 441, 141,  30, 272, 461, 461,  75, 339, 355,
        338, 288, 229, 283, 231,  34,  56,  50, 442,  78, 184,  10, 306, 181,
        167, 142, 180, 375, 372, 132, 371, 361, 261, 509,  27, 168, 334, 266,
        460, 332, 142, 294, 382, 378, 233, 466], device='cuda:0')

In [39]:
target[0][:m_tokens_len[0] + 1].flatten(0)

tensor([390, 144, 282, 301, 452, 260, 102, 294, 480, 233, 267, 275,  12, 419,
        357, 135, 452, 203,  89, 330, 113, 409, 194, 438, 384, 412, 321, 321,
         52,  71, 257, 472, 324, 111, 381, 267, 113, 324, 159, 219, 393, 468,
        471, 234, 275, 412, 393, 310, 316, 512], device='cuda:0')

In [43]:
probs = torch.tensor([0.2, 0.3, 0.5])  # 각 범주에 대한 확률 값
dist = torch.distributions.Categorical(probs)  # Categorical 분포 객체 생성

sample = dist.sample()  # 분포를 따라 샘플링
print(sample)  # 샘플 출력

tensor(1)


In [67]:
m = Categorical(torch.tensor([ 0.25, 0.25, 0.25, 0.25 ]))
y = m.sample()  # equal probability of 0, 1, 2, 3
y

tensor(2)